In [1]:
!pip install selenium

In [152]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import numpy as np

In [33]:
URL='https://main.sci.gov.in/judgments'
DateFrom='01-01-2023'
DateTo='01-03-2023'

In [34]:
driver=webdriver.Firefox()

In [35]:
driver.get(URL)

In [36]:
#Clicking the Date filter button
driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/ul[2]/li[3]/a').click()

In [37]:
#Filling the Captcha
CaptchaValue=driver.find_element(By.XPATH,'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/table/tbody/tr/td[1]/div/p/font').text

In [44]:
CaptchaBox=driver.find_element(By.XPATH,'//*[@id="ansCaptcha"]')

In [45]:
CaptchaBox.clear()

In [46]:
CaptchaBox.send_keys(CaptchaValue)

In [47]:
#Filtering the Respective date
DateFrom_=driver.find_element(By.XPATH,'//*[@id="JBJfrom_date"]')

In [48]:
DateFrom_.clear()

In [49]:
DateFrom_.send_keys(DateFrom)

In [52]:
DateTo_=driver.find_element(By.XPATH,'//*[@id="JBJto_date"]')

In [53]:
DateTo_.clear()

In [55]:
DateTo_.send_keys(DateTo)

In [95]:
driver.find_element(By.XPATH,'//*[@id="v_getJBJ"]').click()

NoSuchElementException: Message: Unable to locate element: /html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[5000]/td[2]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:505:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:135:16


In [57]:
sleep(5)

In [106]:
#Extracting the data from the website
i=0
diaryNo=[]
caseNo=[]
PetitionerName=[]
RespondentName=[]
PetitionersAdvocate=[]
RespondentsAdvocate=[]
Bench=[]
JudgmentBy=[]
while True:
    try:
        diaryNo.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{1+i}]/td[3]').text)
        caseNo.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{2+i}]/td[2]').text)
        PetitionerName.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{3+i}]/td[2]').text)
        RespondentName.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{4+i}]/td[2]').text)
        PetitionersAdvocate.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{5+i}]/td[2]').text)
        RespondentsAdvocate.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{6+i}]/td[2]').text)
        Bench.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{7+i}]/td[2]').text)
        JudgmentBy.append(driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{8+i}]/td[2]').text)
        
        i+=9
    except:
        break

In [146]:
column=['dairy_no','case_no','petitioner_name','respondent_name','petitioner_advocate','respondent_advocate','bench_name','judgment_by']

In [147]:
#Creating a Data Frame with the data
DF=pd.DataFrame([diaryNo,caseNo,PetitionerName,RespondentName,PetitionersAdvocate,RespondentsAdvocate,Bench,JudgmentBy],index=column)

In [148]:
DF=DF.transpose()

In [142]:
#extracting the date and link of the judgment
i=0
Date=[]
Link=[]
while True:
    try:
        Data=driver.find_element(By.XPATH,f'/html/body/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[3]/div/div/table/tbody/tr[{i+2}]/td[3]/a[2]')
        Date.append(Data.text.split(' ')[0])
        Link.append(Data.get_attribute('href'))
        i+=9
    
    except:
        break
        

In [149]:
DF['judgment_link']=Link

In [150]:
DF['judgment_date']=Date

In [158]:
DF=DF.replace(r'^\s*$', np.nan, regex=True)

In [163]:
DF.to_csv('Judgments.csv')